# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)

[Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)


In [ ]:
!pip install -q -U langchain transformers bitsandbytes accelerate

In [ ]:
import numpy as np
import json
from datasets import load_dataset
from random import shuffle
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import gc
from sklearn.metrics import precision_recall_curve

import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

In [5]:
dirname= 'resultsnew'
pickle_dir = 'pickled_results'

def create_dir(name):
    try:
        os.mkdir(name)
    except:
        pass

def save(final_result, name=None, few_shot=False, example=format, data=None):
    create_dir(dirname)
    create_dir(pickle_dir)
    
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    while True:
        file_path = f'{dirname}/{model_name}{inserted_name}_{i}.txt'
        pickle_path = f'{pickle_dir}/{model_name}{inserted_name}_{i}.json'
        try:
            with open(file_path, 'r'):
                i += 1
        except:
            with open(file_path, 'w') as file:
                file.write(model_name+'\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if not isinstance(example, str):
                    example = json.dumps(example)
                file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4))
            if data is None:
                print(f"Warning, {model_name} results data not saving in json, please provide data arg")
            else:
                with open(pickle_path, 'w') as file:
                    file.write(json.dumps(data, indent=4))
            return

### Load the model

Login kako bi se mogli ucitati napredniji modeli

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

Logiranje u huggingiface
treba kljuc, njega nabavite na huggingface stranici

In [ ]:
from huggingface_hub import login
login('insert yours', add_to_git_credential=False, write_permission=True)

BitsAndBytesConfig je novi, to kao kvantizira model pa stane na graficku

In [ ]:

# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
# model_path = 'mistralai/Mistral-7B-v0.3'
# model_path = 'microsoft/phi-2'
# model_path = 'microsoft/Phi-3-mini-128k-instruct'
# model_path = 'microsoft/Phi-3-mini-4k-instruct'
# model_path = '01-ai/Yi-1.5-6B'
# model_path = 'netcat420/MFANN3bv0.6'
model_path = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_path = "meta-llama/Meta-Llama-3-8B"
# model_path = "allenai/OLMo-7B-Instruct"
# model_path = "tiiuae/falcon-7b-instruct"
# model_path = "tiiuae/falcon-7b"
# model_path = 'Intel/neural-chat-7b-v3-1'
# model_path = "google/gemma-1.1-7b-it"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def get_mistral():
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True,
    )
    return tokenizer, model

def get_llama():
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model_4bit = AutoModelForCausalLM.from_pretrained(
        model_path,
#         "SweatyCrayfish/llama-3-8b-quantized", 
#         load_in_4bit=True, 
#         torch_dtype=torch.float16,
        device_map="auto",
        quantization_config=quantization_config,
    )
    return tokenizer, model_4bit

tokenizer, model = get_mistral()


### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`, ako na Kaggleu, onda napraviti svoj dataset te ga dodati pod input

In [ ]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
data_path = '/kaggle/input/brainteaser/data'
sentence_data_path = f"{data_path}/SP-train.npy"
wordplay_data_list = f"{data_path}/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

In [38]:
def get_format(letters, demonstration):
    newline = '\n'
    format = f"""Question: {"{}"}

Choices:
{''.join('(' + x + ') {}' + newline for x in letters)}"""
    if demonstration is not None:
        format = demonstration + "\n\n" + format
    format += "\nAnswer:("
    return format

def get_prompt(sample: dict, *args, **kwargs):
    format = get_format(*args, **kwargs)
    sample_demo = format.format(
        sample['question'], *sample['choice_list'])
    return sample_demo

In [39]:
def evaluate(model, samples: list, letters=None, demonstration=None):
    if letters is None:
        letters = ["A", "B", "C", "D"]
    for sample in tqdm(samples):
        text = get_prompt(sample, letters=letters, demonstration=demonstration)
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(model.device)
        with torch.no_grad():
            results = model(inputs)
        probs = torch.nn.functional.softmax(results.logits.squeeze()[-1], dim=-1)
        model_answer = tokenizer.decode(probs.argmax())
        model_answer_prob= float(probs.max())
        model_correct = model_answer == letters[sample["label"]]
        sample['model_probs'] = probs
        sample['model_answer'] = model_answer
        sample['model_answer_prob'] = model_answer_prob
        sample['model_correct'] = model_correct

In [ ]:
letters = ["A", "B", "C", "D"]
demonstration = "The following question is a brainteaser."
print(get_prompt(test_data_list[0], letters=letters, demonstration=demonstration))
evaluate(model, test_data_list, letters=letters, demonstration=demonstration)

In [ ]:
samples = test_data_list
y_true = [sample["model_answer"] for sample in samples]
pos_label = [letters[sample["label"]] for sample in samples]
y_prob = [float(sample["model_answer_prob"]) for sample in samples]


model_name = model.name_or_path.split('/')[-1]
precision, recall, thresholds = precision_recall_curve(y_true, y_prob, pos_label=pos_label)

plt.fill_between(recall, precision)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title(f"Precision-Recall curve {model_name}");

# plt.fill_between(thresholds, precision[:-1])
# plt.ylabel("Precision")
# plt.xlabel("Tresholds")
# plt.xlim(0,1)
# plt.title(f"Treshold-precision curve {model_name}");